In [0]:
!ls
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

kaggle.json  sample_data


In [0]:
!kaggle datasets download -d weixinxu666/digit-recognizer

 59% 9.00M/15.3M [00:01<00:01, 5.82MB/s]
100% 15.3M/15.3M [00:01<00:00, 9.35MB/s]


In [0]:
!unzip \*.zip  && rm *.zip


Archive:  digit-recognizer.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [0]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.keras.backend.clear_session()  # For easy reset of notebook state.


print("[INFO] Data loading..")
Raw_train = pd.read_csv('train.csv')
Raw_test = pd.read_csv('test.csv')


trainY = np.array(Raw_train['label'])
trainX = np.array(Raw_train.iloc[:, 1:]) / 255.0
trainX = np.reshape(trainX, (len(trainX), 28, 28, 1))

print("[INFO] Train data size: " + str(trainX.shape))

inputs = keras.Input(shape=(28,28, 1))
x = layers.Conv2D(32, kernel_size=(3,3), padding='same', activation='relu')(inputs)
x = layers.Conv2D(32, kernel_size=(3,3), padding='same', activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation='softmax')(x)


model = keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')


model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

history = model.fit(trainX, trainY,
                    batch_size=128,
                    epochs=100,
                    validation_split=0.2)



[INFO] Data loading..
[INFO] Train data size: (42000, 28, 28, 1)
Epoch 1/30
263/263 [==============================] - 6s 23ms/step - loss: 1.6095 - accuracy: 0.8538 - val_loss: 1.4950 - val_accuracy: 0.9668
Epoch 2/30
263/263 [==============================] - 5s 21ms/step - loss: 1.5139 - accuracy: 0.9478 - val_loss: 1.4938 - val_accuracy: 0.9674
Epoch 3/30
263/263 [==============================] - 5s 21ms/step - loss: 1.5000 - accuracy: 0.9617 - val_loss: 1.4835 - val_accuracy: 0.9775
Epoch 4/30
263/263 [==============================] - 5s 20ms/step - loss: 1.4936 - accuracy: 0.9678 - val_loss: 1.4803 - val_accuracy: 0.9808
Epoch 5/30
263/263 [==============================] - 5s 20ms/step - loss: 1.4924 - accuracy: 0.9688 - val_loss: 1.4823 - val_accuracy: 0.9787
Epoch 6/30
263/263 [==============================] - 5s 20ms/step - loss: 1.4891 - accuracy: 0.9722 - val_loss: 1.4786 - val_accuracy: 0.9826
Epoch 7/30
263/263 [==============================] - 5s 21ms/step - loss: 1.

In [0]:
# predict results
Raw_test = np.reshape(np.array(Raw_test), (len(Raw_test), 28, 28, 1))
results = model.predict(Raw_test)

# select the indix with the maximum probability
results = np.argmax(results, axis=1)

results = pd.Series(results,name="Label")

submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("mySubmission.csv",index=False)